In [1]:
import nba_api.stats.endpoints as nba
import nba_api as static
import config
import pandas as pd
import mysql_helper
import mysql.connector

In [2]:
dbname = "nba"
cnx = mysql.connector.connect(
    host = config.host,
    user = config.user,
    passwd = config.password,
    db = dbname)
cursor = cnx.cursor(buffered=True)

In [3]:
stmt = """SELECT GAME_ID from nba.box_score"""
cursor.execute(stmt)


In [4]:
ids = cursor.fetchall()

In [5]:
ids_df = pd.DataFrame(ids)

In [6]:
ids_df.columns = ['game_id']

In [7]:
ids_df.head()
season_ids = {'00217':'22017',
             '00218':'22018',
             '00219':'22019',}
ids_df['SEASON_ID'] = ids_df['game_id'].map(lambda x : season_ids[x[0:5]])

In [8]:
ids_df.head()

,game_id,SEASON_ID
0,0021700001,22017
1,0021700001,22017
2,0021700001,22017
3,0021700001,22017
4,0021700001,22017


In [21]:
stmt = """ALTER TABLE nba.box_score
        ADD COLUMN TD INT(1)"""
cursor.execute(stmt)

In [24]:
stmt = """SELECT * FROM nba.box_score"""
cursor.execute(stmt)

In [25]:
box_scores = cursor.fetchall()

In [26]:
field_names = [i[0] for i in cursor.description]
box_scores_df = pd.DataFrame(box_scores, columns=field_names)

In [27]:
box_scores_df.head()

,GAME_ID,TEAM_ID,TEAM_ABBREVIATION,TEAM_CITY,PLAYER_ID,PLAYER_NAME,START_POSITION,COMMENT,MIN,FGM,...,BLK,PF,PTS,PLUS_MINUS,pk,TOS,SEASON_ID,DK_PTS,DD,TD
0,None,NaN,None,None,NaN,None,None,None,None,NaN,...,NaN,NaN,NaN,NaN,,NaN,22017,None,None,None
1,0021700001,1.610613e+09,CLE,Cleveland,101112.0,Channing Frye,,DNP - Coach's Decision,None,NaN,...,NaN,NaN,NaN,NaN,0021700001101112,NaN,22017,None,None,None
2,0021700001,1.610613e+09,CLE,Cleveland,101181.0,Jose Calderon,,DNP - Coach's Decision,None,NaN,...,NaN,NaN,NaN,NaN,0021700001101181,NaN,22017,None,None,None
3,0021700001,1.610613e+09,BOS,Boston,1626179.0,Terry Rozier,,,19:32,2.0,...,0.0,0.0,9.0,5.0,00217000011626179,0.0,22017,None,None,None
4,0021700001,1.610613e+09,CLE,Cleveland,1626224.0,Cedi Osman,,DNP - Coach's Decision,None,NaN,...,NaN,NaN,NaN,NaN,00217000011626224,NaN,22017,None,None,None


In [37]:
box_clean = box_scores_df[box_scores_df["MIN"].notnull()]

In [39]:
box_clean.columns

Index(['GAME_ID', 'TEAM_ID', 'TEAM_ABBREVIATION', 'TEAM_CITY', 'PLAYER_ID',
       'PLAYER_NAME', 'START_POSITION', 'COMMENT', 'MIN', 'FGM', 'FGA',
       'FG_PCT', 'FG3M', 'FG3A', 'FG3_PCT', 'FTM', 'FTA', 'FT_PCT', 'OREB',
       'DREB', 'REB', 'AST', 'STL', 'BLK', 'PF', 'PTS', 'PLUS_MINUS', 'pk',
       'TOS', 'SEASON_ID', 'DK_PTS', 'DD', 'TD'],
      dtype='object')

In [ ]:
for i in range(0, len(box_clean)):
    doubles = int(box_clean.iloc[i]['PTS'] > 9) + int(box_clean.iloc[i]['AST']> 9) + int(box_clean.iloc[i]['REB'] > 9) + int(box_clean.iloc[i]['STL'] > 9) + int(box_clean.iloc[i]['BLK'] > 9)
    if doubles == 2:
        box_clean.iloc[i]['DD'] = 1
        box_clean.iloc[i]['TD'] = 0
    elif doubles > 2:
        box_clean.iloc[i]['DD'] = 0
        box_clean.iloc[i]['TD'] = 1
    else:
        box_clean.iloc[i]['DD'] = 0
        box_clean.iloc[i]['TD'] = 1

/Users/marshall132/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  # Remove the CWD from sys.path while we load stuff.
/Users/marshall132/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  # This is added back by InteractiveShellApp.init_path()
/Users/marshall132/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-cop

In [42]:
box_clean.head()

,GAME_ID,TEAM_ID,TEAM_ABBREVIATION,TEAM_CITY,PLAYER_ID,PLAYER_NAME,START_POSITION,COMMENT,MIN,FGM,...,PF,PTS,PLUS_MINUS,pk,TOS,SEASON_ID,DK_PTS,DD,TD,doubles
3,0021700001,1.610613e+09,BOS,Boston,1626179.0,Terry Rozier,,,19:32,2.0,...,0.0,9.0,5.0,00217000011626179,0.0,22017,None,None,None,False
5,0021700001,1.610613e+09,BOS,Boston,1627759.0,Jaylen Brown,G,,39:36,11.0,...,5.0,25.0,-5.0,00217000011627759,3.0,22017,None,None,None,True
7,0021700001,1.610613e+09,BOS,Boston,1628369.0,Jayson Tatum,F,,36:32,5.0,...,4.0,14.0,5.0,00217000011628369,1.0,22017,None,None,None,True
8,0021700001,1.610613e+09,BOS,Boston,1628400.0,Semi Ojeleye,,,8:39,0.0,...,1.0,0.0,-9.0,00217000011628400,0.0,22017,None,None,None,False
10,0021700001,1.610613e+09,BOS,Boston,201143.0,Al Horford,C,,32:07,2.0,...,2.0,9.0,8.0,0021700001201143,0.0,22017,None,None,None,False


In [14]:
ids_df['SEASON_ID'].value_counts()

22017    30980
22018    30808
22019     6029
Name: SEASON_ID, dtype: int64

In [11]:
season_id_tuples = [(x,) for x in ids_df['SEASON_ID'].values]

In [12]:
season_id_tuples

[('22017',),
 ('22017',),
 ('22017',),
 ('22017',),
 ('22017',),
 ('22017',),
 ('22017',),
 ('22017',),
 ('22017',),
 ('22017',),
 ('22017',),
 ('22017',),
 ('22017',),
 ('22017',),
 ('22017',),
 ('22017',),
 ('22017',),
 ('22017',),
 ('22017',),
 ('22017',),
 ('22017',),
 ('22017',),
 ('22017',),
 ('22017',),
 ('22017',),
 ('22017',),
 ('22017',),
 ('22017',),
 ('22017',),
 ('22017',),
 ('22017',),
 ('22017',),
 ('22017',),
 ('22017',),
 ('22017',),
 ('22017',),
 ('22017',),
 ('22017',),
 ('22017',),
 ('22017',),
 ('22017',),
 ('22017',),
 ('22017',),
 ('22017',),
 ('22017',),
 ('22017',),
 ('22017',),
 ('22017',),
 ('22017',),
 ('22017',),
 ('22017',),
 ('22017',),
 ('22017',),
 ('22017',),
 ('22017',),
 ('22017',),
 ('22017',),
 ('22017',),
 ('22017',),
 ('22017',),
 ('22017',),
 ('22017',),
 ('22017',),
 ('22017',),
 ('22017',),
 ('22017',),
 ('22017',),
 ('22017',),
 ('22017',),
 ('22017',),
 ('22017',),
 ('22017',),
 ('22017',),
 ('22017',),
 ('22017',),
 ('22017',),
 ('22017',),

In [15]:
stmt = """UPDATE nba.box_score SET 
    SEASON_ID = SELECT game_logs.SEASON_ID 
    FROM nba.box_score 
    INNER JOIN nba.game_logs ON nba.box_score.GAME_ID = nba.game_logs.GAME_ID
"""
cursor.execute(stmt)

ProgrammingError: 1064 (42000): You have an error in your SQL syntax; check the manual that corresponds to your MySQL server version for the right syntax to use near 'SELECT game_logs.SEASON_ID 
    FROM nba.box_score 
    INNER JOIN nba.game_logs' at line 2

In [18]:
stmt="""UPDATE nba.box_score
    SET SEASON_ID = "22019"
    WHERE SUBSTRING(GAME_ID, 1,5) = "00219"
    """
cursor.execute(stmt)
cnx.commit()

In [2]:
career = nba.PlayerCareerStats(player_id='203076')
career.get_data_frames()[0]

,PLAYER_ID,SEASON_ID,LEAGUE_ID,TEAM_ID,TEAM_ABBREVIATION,PLAYER_AGE,GP,GS,MIN,FGM,...,FT_PCT,OREB,DREB,REB,AST,STL,BLK,TOV,PF,PTS
0,203076,2012-13,00,1610612740,NOH,20.0,64,60,1846.0,349,...,0.751,165,357,522,63,75,112,89,158,867
1,203076,2013-14,00,1610612740,NOP,21.0,67,66,2358.0,522,...,0.791,207,466,673,105,89,189,109,200,1394
2,203076,2014-15,00,1610612740,NOP,22.0,68,68,2455.0,642,...,0.805,173,523,696,149,100,200,95,141,1656
3,203076,2015-16,00,1610612740,NOP,23.0,61,61,2164.0,560,...,0.758,130,497,627,116,78,125,121,148,1481
4,203076,2016-17,00,1610612740,NOP,24.0,75,75,2708.0,770,...,0.802,172,712,884,157,94,167,181,168,2099
5,203076,2017-18,00,1610612740,NOP,25.0,75,75,2727.0,780,...,0.828,187,644,831,174,115,193,162,159,2110
6,203076,2018-19,00,1610612740,NOP,26.0,56,56,1850.0,530,...,0.794,174,498,672,218,88,135,112,132,1452
7,203076,2019-20,00,1610612747,LAL,26.0,15,15,524.0,132,...,0.878,34,98,132,54,22,45,43,36,383


In [40]:
flatiron = mysql_helper.Connection(config.host, config.user, config.password)

In [41]:
nba_db = mysql_helper.DataBase(flatiron, 'nba')

In [5]:
player_df = nba.CommonAllPlayers().get_data_frames()[0]

In [6]:
player_df.columns

Index(['PERSON_ID', 'DISPLAY_LAST_COMMA_FIRST', 'DISPLAY_FIRST_LAST',
       'ROSTERSTATUS', 'FROM_YEAR', 'TO_YEAR', 'PLAYERCODE', 'TEAM_ID',
       'TEAM_CITY', 'TEAM_NAME', 'TEAM_ABBREVIATION', 'TEAM_CODE',
       'GAMES_PLAYED_FLAG', 'OTHERLEAGUE_EXPERIENCE_CH'],
      dtype='object')

In [7]:
player_df[player_df['DISPLAY_FIRST_LAST']== 'Joel Embiid']['PERSON_ID']

1147    203954
Name: PERSON_ID, dtype: int64

In [8]:
embiid_log =nba.PlayerGameLog(player_id='203954').get_data_frames()[0]

In [9]:
embiid_log.head()

,SEASON_ID,Player_ID,Game_ID,GAME_DATE,MATCHUP,WL,MIN,FGM,FGA,FG_PCT,...,DREB,REB,AST,STL,BLK,TOV,PF,PTS,PLUS_MINUS,VIDEO_AVAILABLE
0,22019,203954,0021900230,"NOV 23, 2019",PHI vs. MIA,W,26,8,15,0.533,...,8,11,2,2,2,2,2,23,28,1
1,22019,203954,0021900218,"NOV 22, 2019",PHI vs. SAS,W,28,9,13,0.692,...,11,14,3,1,1,4,4,21,13,1
2,22019,203954,0021900202,"NOV 20, 2019",PHI vs. NYK,W,34,7,19,0.368,...,10,12,5,0,1,3,3,23,10,1
3,22019,203954,0021900183,"NOV 17, 2019",PHI @ CLE,W,23,6,13,0.462,...,3,4,2,1,1,2,3,14,14,1
4,22019,203954,0021900172,"NOV 15, 2019",PHI @ OKC,L,38,12,24,0.500,...,10,12,2,1,0,1,4,31,-10,1


In [10]:
embiid_log.columns

Index(['SEASON_ID', 'Player_ID', 'Game_ID', 'GAME_DATE', 'MATCHUP', 'WL',
       'MIN', 'FGM', 'FGA', 'FG_PCT', 'FG3M', 'FG3A', 'FG3_PCT', 'FTM', 'FTA',
       'FT_PCT', 'OREB', 'DREB', 'REB', 'AST', 'STL', 'BLK', 'TOV', 'PF',
       'PTS', 'PLUS_MINUS', 'VIDEO_AVAILABLE'],
      dtype='object')

In [11]:
nba.ScoreboardV2(game_date='11-24-2019').get_data_frames()[0]

,GAME_DATE_EST,GAME_SEQUENCE,GAME_ID,GAME_STATUS_ID,GAME_STATUS_TEXT,GAMECODE,HOME_TEAM_ID,VISITOR_TEAM_ID,SEASON,LIVE_PERIOD,LIVE_PC_TIME,NATL_TV_BROADCASTER_ABBREVIATION,HOME_TV_BROADCASTER_ABBREVIATION,AWAY_TV_BROADCASTER_ABBREVIATION,LIVE_PERIOD_TIME_BCAST,ARENA_NAME,WH_STATUS
0,2019-11-24T00:00:00,1,0021900235,2,Halftime,20191124/DALHOU,1610612745,1610612742,2019,2,,None,ATTSN-SW,FSSW-DAL,Q2 -,Toyota Center,0
1,2019-11-24T00:00:00,2,0021900236,1,6:00 pm ET,20191124/BKNNYK,1610612752,1610612751,2019,0,,None,MSG,YES,Q0 -,Madison Square Garden,0
2,2019-11-24T00:00:00,3,0021900237,1,6:00 pm ET,20191124/SACWAS,1610612764,1610612758,2019,0,,None,NBCSWA,NBCSCA,Q0 -,Capital One Arena,0
3,2019-11-24T00:00:00,4,0021900238,1,8:00 pm ET,20191124/PHXDEN,1610612743,1610612756,2019,0,,None,ALT,FSA,Q0 -,Pepsi Center,0
4,2019-11-24T00:00:00,5,0021900239,1,9:00 pm ET,20191124/NOPLAC,1610612746,1610612740,2019,0,,None,FSPT,FSNO,Q0 -,Staples Center,0


In [12]:
game_logs = nba.LeagueGameLog(season='2019-20').get_data_frames()[0]

In [13]:
game_logs.head()

,SEASON_ID,TEAM_ID,TEAM_ABBREVIATION,TEAM_NAME,GAME_ID,GAME_DATE,MATCHUP,WL,MIN,FGM,...,DREB,REB,AST,STL,BLK,TOV,PF,PTS,PLUS_MINUS,VIDEO_AVAILABLE
0,22019,1610612747,LAL,Los Angeles Lakers,0021900002,2019-10-22,LAL @ LAC,L,240,37,...,32,41,20,4,7,15,24,102,-10,1
1,22019,1610612746,LAC,LA Clippers,0021900002,2019-10-22,LAC vs. LAL,W,240,42,...,34,45,24,8,5,14,25,112,10,1
2,22019,1610612740,NOP,New Orleans Pelicans,0021900001,2019-10-22,NOP @ TOR,L,265,43,...,37,53,30,4,9,19,34,122,-8,1
3,22019,1610612761,TOR,Toronto Raptors,0021900001,2019-10-22,TOR vs. NOP,W,265,42,...,41,57,23,7,3,17,24,130,8,1
4,22019,1610612738,BOS,Boston Celtics,0021900008,2019-10-23,BOS @ PHI,L,240,33,...,31,41,18,4,2,11,29,93,-14,1


In [14]:
def get_game_logs(year):
    next_ = str(int(year)+1)[-2]
    df = nba.LeagueGameLog(season=year + '-' + next_).get_data_frames()[0]
    return df

In [15]:
seasons = ['2017', '2018', '2019']
dfs = []
for season in seasons:
    dfs.append(get_game_logs(season))
total_logs = pd.concat(dfs)

In [16]:
total_logs.tail()

,SEASON_ID,TEAM_ID,TEAM_ABBREVIATION,TEAM_NAME,GAME_ID,GAME_DATE,MATCHUP,WL,MIN,FGM,...,DREB,REB,AST,STL,BLK,TOV,PF,PTS,PLUS_MINUS,VIDEO_AVAILABLE
465,22019,1610612763,MEM,Memphis Grizzlies,0021900232,2019-11-23,MEM vs. LAL,L,240,43,...,35,47,28,12,3,22,19,108,-1,1
466,22019,1610612756,PHX,Phoenix Suns,0021900225,2019-11-23,PHX @ MIN,W,240,33,...,44,56,22,9,3,7,17,100,2,1
467,22019,1610612750,MIN,Minnesota Timberwolves,0021900225,2019-11-23,MIN vs. PHX,L,240,38,...,48,57,26,2,10,14,23,98,-2,1
468,22019,1610612742,DAL,Dallas Mavericks,0021900235,2019-11-24,DAL @ HOU,None,120,29,...,18,28,16,6,1,8,14,78,18,0
469,22019,1610612745,HOU,Houston Rockets,0021900235,2019-11-24,HOU vs. DAL,None,120,21,...,17,23,13,4,0,8,12,60,-18,0


In [17]:
total_logs['pk'] = total_logs['GAME_ID']+total_logs['TEAM_ABBREVIATION']

In [18]:
total_logs.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 5390 entries, 0 to 469
Data columns (total 30 columns):
SEASON_ID            5390 non-null object
TEAM_ID              5390 non-null int64
TEAM_ABBREVIATION    5390 non-null object
TEAM_NAME            5390 non-null object
GAME_ID              5390 non-null object
GAME_DATE            5390 non-null object
MATCHUP              5390 non-null object
WL                   5388 non-null object
MIN                  5390 non-null int64
FGM                  5390 non-null int64
FGA                  5390 non-null int64
FG_PCT               5390 non-null float64
FG3M                 5390 non-null int64
FG3A                 5390 non-null int64
FG3_PCT              5390 non-null float64
FTM                  5390 non-null int64
FTA                  5390 non-null int64
FT_PCT               5390 non-null float64
OREB                 5390 non-null int64
DREB                 5390 non-null int64
REB                  5390 non-null int64
AST                 

In [19]:
nba_db.table_fromDf(total_logs,'game_logs', 'pk')

CREATE TABLE IF NOT EXISTS game_logs (SEASON_ID VARCHAR (250), TEAM_ID INT (64), TEAM_ABBREVIATION VARCHAR (250), TEAM_NAME VARCHAR (250), GAME_ID VARCHAR (250), GAME_DATE VARCHAR (250), MATCHUP VARCHAR (250), WL VARCHAR (250), MIN INT (64), FGM INT (64), FGA INT (64), FG_PCT FLOAT, FG3M INT (64), FG3A INT (64), FG3_PCT FLOAT, FTM INT (64), FTA INT (64), FT_PCT FLOAT, OREB INT (64), DREB INT (64), REB INT (64), AST INT (64), STL INT (64), BLK INT (64), TOV INT (64), PF INT (64), PTS INT (64), PLUS_MINUS INT (64), VIDEO_AVAILABLE INT (64), pk VARCHAR (250), PRIMARY KEY (pk));
Creating a new table
OK


In [ ]:
nba_db.insert_fromDf_iteration(total_logs, 'game_logs')

In [21]:
select_statement = """SELECT GAME_ID, GAME_DATE FROM nba.game_logs WHERE DATE_FORMAT(GAME_DATE, '%Y-%m-%d') < DATE_FORMAT('2018-01-01', '%Y-%m-%d');"""
cursor.execute(select_statement)
game_ids = cursor.fetchall()

In [25]:

game_ids = set(game_ids)
# game_ids_df = pd.DataFrame(game_ids)

In [28]:
print(game_ids)

{('0021700432', '2017-12-16'), ('0021700158', '2017-11-07'), ('0021700480', '2017-12-23'), ('0021700307', '2017-11-29'), ('0021700164', '2017-11-09'), ('0021700034', '2017-10-21'), ('0021700092', '2017-10-30'), ('0021700421', '2017-12-15'), ('0021700414', '2017-12-14'), ('0021700542', '2017-12-31'), ('0021700071', '2017-10-27'), ('0021700156', '2017-11-07'), ('0021700369', '2017-12-08'), ('0021700040', '2017-10-22'), ('0021700053', '2017-10-24'), ('0021700412', '2017-12-13'), ('0021700002', '2017-10-17'), ('0021700296', '2017-11-27'), ('0021700013', '2017-10-18'), ('0021700446', '2017-12-18'), ('0021700514', '2017-12-27'), ('0021700062', '2017-10-25'), ('0021700479', '2017-12-22'), ('0021700093', '2017-10-30'), ('0021700528', '2017-12-29'), ('0021700195', '2017-11-13'), ('0021700415', '2017-12-14'), ('0021700168', '2017-11-09'), ('0021700426', '2017-12-15'), ('0021700433', '2017-12-16'), ('0021700255', '2017-11-22'), ('0021700481', '2017-12-23'), ('0021700043', '2017-10-23'), ('0021700

In [32]:
pd.DataFrame(game_ids)

ValueError: DataFrame constructor not properly called!

In [35]:
box_score = nba.BoxScoreTraditionalV2(game_id='0021900002').get_data_frames()[0]

In [37]:
box_score['pk'] = str(box_score['PLAYER_ID']) + box_score['GAME_ID']

In [43]:
box_score['TOS'] = box_score['TO']

In [47]:
box_score.drop('TO', axis=1, inplace=True)

In [49]:
box_score.columns

Index(['GAME_ID', 'TEAM_ID', 'TEAM_ABBREVIATION', 'TEAM_CITY', 'PLAYER_ID',
       'PLAYER_NAME', 'START_POSITION', 'COMMENT', 'MIN', 'FGM', 'FGA',
       'FG_PCT', 'FG3M', 'FG3A', 'FG3_PCT', 'FTM', 'FTA', 'FT_PCT', 'OREB',
       'DREB', 'REB', 'AST', 'STL', 'BLK', 'PF', 'PTS', 'PLUS_MINUS', 'pk',
       'TOS'],
      dtype='object')

In [50]:
nba_db.table_fromDf(box_score,'box_score', 'pk')

CREATE TABLE IF NOT EXISTS box_score (GAME_ID VARCHAR (250), TEAM_ID INT (64), TEAM_ABBREVIATION VARCHAR (250), TEAM_CITY VARCHAR (250), PLAYER_ID INT (64), PLAYER_NAME VARCHAR (250), START_POSITION VARCHAR (250), COMMENT VARCHAR (250), MIN VARCHAR (250), FGM FLOAT, FGA FLOAT, FG_PCT FLOAT, FG3M FLOAT, FG3A FLOAT, FG3_PCT FLOAT, FTM FLOAT, FTA FLOAT, FT_PCT FLOAT, OREB FLOAT, DREB FLOAT, REB FLOAT, AST FLOAT, STL FLOAT, BLK FLOAT, PF FLOAT, PTS FLOAT, PLUS_MINUS FLOAT, pk VARCHAR (250), TOS FLOAT, PRIMARY KEY (pk));
Creating a new table
OK


In [22]:
box_score.columns

Index(['GAME_ID', 'TEAM_ID', 'TEAM_ABBREVIATION', 'TEAM_CITY', 'PLAYER_ID',
       'PLAYER_NAME', 'START_POSITION', 'COMMENT', 'MIN', 'FGM', 'FGA',
       'FG_PCT', 'FG3M', 'FG3A', 'FG3_PCT', 'FTM', 'FTA', 'FT_PCT', 'OREB',
       'DREB', 'REB', 'AST', 'STL', 'BLK', 'TO', 'PF', 'PTS', 'PLUS_MINUS'],
      dtype='object')

In [7]:
def get_box_score(id):
    box_score = nba.BoxScoreTraditionalV2(game_id=id).get_data_frames()[0]
    box_score['pk'] = str(box_score['PLAYER_ID']) + box_score['GAME_ID']
    

In [ ]:
dfs = []
for game in game_ids:
    dfs.append(get_box_score(game))
frame = pd.concat(dfs)

In [23]:
## Use scoreboard to get game ids for each game for past 3 years and each sequential day,
## Then use game id to feed into gamelog to get team box scores for each day
## Feed game ids into boxscore traditional to get subsequent box scores for each player
